In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [34]:
# cd /content/drive/My Drive/'Colab Notebooks'/
cd /content/drive/My Drive/Walmart_Competition 

/content/drive/My Drive/Walmart_Competition


In [35]:
ls

features.csv                                        sampleSubmission.csv
Keras_pickle_model.pkl                              Scikit_model.ipynb
Linear_Regression_NN.ipynb                          stores.csv
LinearRegression_Regularization_Dropout_Adam.ipynb  test.csv
ML_models.ipynb                                     train.csv
RF_pickle_model.pkl                                 Walmart_Comp.ipynb


In [0]:
# Imports
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Sklearn Imports
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import xgboost as xgb

In [0]:
# Keras Imports
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [0]:
df_features = pd.read_csv('features.csv')
df_stores = pd.read_csv('stores.csv')
df_test = df_test_for_submission = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

In [0]:
data_frame = pd.merge(df_train,df_features,on=['Store','Date','IsHoliday'])
data_frame_test = pd.merge(df_test,df_features,on=['Store','Date','IsHoliday'])

In [0]:
data_frame = pd.merge(data_frame,df_stores,on='Store')
data_frame_test = pd.merge(data_frame_test,df_stores,on='Store')

In [0]:
data_frame.fillna(0,inplace=True)
data_frame_test.fillna(0,inplace=True)

In [43]:
print(data_frame.isna().sum())
print("-----------------------")
print(data_frame_test.isna().sum())

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
dtype: int64
-----------------------
Store           0
Dept            0
Date            0
IsHoliday       0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
dtype: int64


In [0]:
data_frame["Weekly_Sales"] = np.where(data_frame["Weekly_Sales"] < 0,0,data_frame["Weekly_Sales"])
data_frame["MarkDown2"] = np.where(data_frame["MarkDown2"] < 0,0,data_frame["MarkDown2"])
data_frame["MarkDown3"] = np.where(data_frame["MarkDown3"] < 0,0,data_frame["MarkDown3"])

data_frame_test["MarkDown2"] = np.where(data_frame_test["MarkDown2"] < 0,0,data_frame_test["MarkDown2"])
data_frame_test["MarkDown3"] = np.where(data_frame_test["MarkDown3"] < 0,0,data_frame_test["MarkDown3"])

In [45]:
print(data_frame["Type"].unique())
print(data_frame_test["Type"].unique())

['A' 'B' 'C']
['A' 'B' 'C']


In [0]:
data_frame["Date"] = pd.to_datetime(data_frame["Date"])
data_frame["Year"] = data_frame["Date"].dt.year
data_frame["Month"] = data_frame["Date"].dt.month
data_frame["Day"] = data_frame["Date"].dt.day

data_frame_test["Date"] = pd.to_datetime(data_frame_test["Date"])
data_frame_test["Year"] = data_frame_test["Date"].dt.year
data_frame_test["Month"] = data_frame_test["Date"].dt.month
data_frame_test["Day"] = data_frame_test["Date"].dt.day

In [0]:
data_frame = data_frame.drop("Date",axis=1)
data_frame_test = data_frame_test.drop("Date",axis=1)

In [0]:
data_frame["IsHoliday"] = data_frame["IsHoliday"].astype(int)  # False = 0, True = 1
data_frame_test["IsHoliday"] = data_frame_test["IsHoliday"].astype(int)

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_frame["Type"] = le.fit_transform(data_frame["Type"])
data_frame_test["Type"] = le.fit_transform(data_frame_test["Type"])

In [0]:
y = data_frame["Weekly_Sales"].values
X = data_frame.drop(["Weekly_Sales","IsHoliday","CPI","Unemployment","MarkDown1","Year"],axis=1)
wmae_feature = data_frame["IsHoliday"].values
X_test = data_frame_test.drop(["IsHoliday","CPI","Unemployment","MarkDown1","Year"],axis=1)

In [51]:
#y = data_frame["Weekly_Sales"].values
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
min_max_X = MinMaxScaler()
min_max_y = MinMaxScaler()

X = min_max_X.fit_transform(X)
X_test = min_max_X.transform(X_test)
y = y.reshape(-1,1)
y = min_max_y.fit_transform(y)

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)


print(X.shape,y.shape)
print(X_test.shape)

(421570, 12) (421570, 1)
(115064, 12)


In [0]:
# Function of all the Models I have experimented on
def Linear_Regression_model(X_train, y_train,X_val,y_val):
  
  LR = LinearRegression()
  LR.fit(X_train, y_train)
  score_train_acc = LR.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = LR.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_LR = LR.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_LR))
  print("MSE value: ",mean_squared_error(y_val,y_pred_LR))
  
  
  pkl_filename = "LR_pickle_model.pkl"
  
  with open(pkl_filename, 'wb') as file:
    pickle.dump(LR, file)
  return score_train_acc, score_val_acc

def Linear_Regression_Ridge_model(X_train, y_train,X_val,y_val):
  
  LR_R = Ridge()
  parameters = {'alpha':[0.1, 0.001, 1]}
  LR_R = GridSearchCV(LR_R, param_grid=parameters ,cv=5)
  LR_R.fit(X_train, y_train)
  score_train_acc = LR_R.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = LR_R.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_LR = LR_R.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_LR))
  print("MSE value: ",mean_squared_error(y_val,y_pred_LR))
  print("Best Params", LR_R.best_params_)
  
  
  pkl_filename = "LR_R_pickle_model.pkl"
  
  with open(pkl_filename, 'wb') as file:
    pickle.dump(LR_R, file)
  return score_train_acc, score_val_acc

def Linear_Regression_Lasso_model(X_train, y_train,X_val,y_val):
  
  LR_L = Lasso()
  parameters = {'alpha':[0.1, 0.001, 1]}
  LR_L = GridSearchCV(LR_L, param_grid=parameters ,cv=5)
  LR_L.fit(X_train, y_train)
  score_train_acc = LR_L.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = LR_L.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_LR = LR_L.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_LR))
  print("MSE value: ",mean_squared_error(y_val,y_pred_LR))
  print("Best Params", LR_L.best_params_)
  
  
  pkl_filename = "LR_L_pickle_model.pkl"
  
  with open(pkl_filename, 'wb') as file:
    pickle.dump(LR_L, file)
  return score_train_acc, score_val_acc

def Linear_Regression_ElasticNet_model(X_train, y_train,X_val,y_val):
  
  LR_E = ElasticNet()
  parameters = {'alpha':[0.1, 0.001, 1],'l1_ratio':[0.2,0.3,0.4,0.5]}
  LR_E = GridSearchCV(LR_E, param_grid=parameters ,cv=5)
  LR_E.fit(X_train, y_train)
  score_train_acc = LR_E.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = LR_E.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_LR = LR_E.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_LR))
  print("MSE value: ",mean_squared_error(y_val,y_pred_LR))
  print("Best Params", LR_E.best_params_)
  
  pkl_filename = "LR_E_pickle_model.pkl"
  
  with open(pkl_filename, 'wb') as file:
    pickle.dump(LR_E, file)
  return score_train_acc, score_val_acc

def RF_model(X_train, y_train,X_val,y_val):
  RF = RandomForestRegressor(n_estimators=50,criterion='mse')
  RF.fit(X_train, y_train)
  score_train_acc = RF.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = RF.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_RF = RF.predict(X_val)
  print("Y_pred_RF values",y_pred_RF.max())
  print("R2 score: ",r2_score(y_val,y_pred_RF))
  print("MSE value: ",np.sqrt(mean_squared_error(y_val,y_pred_RF)))
  
  pkl_filename = "RF_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(RF, file)
  return score_train_acc, score_val_acc,y_pred_RF

def GB_model(X_train, y_train,X_val,y_val):
  GB = GradientBoostingRegressor(n_estimators=500,criterion='mse')
  GB.fit(X_train, y_train)
  score_train_acc = GB.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = GB.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_GB = GB.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_GB))
  print("MSE value: ",mean_squared_error(y_val,y_pred_GB))
  
  pkl_filename = "GB_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(GB, file)
  return score_train_acc, score_val_acc

def XGBoost_model(X_train, y_train,X_val,y_val):
  XGB = xgb.XGBRegressor(booster='gbtree', colsample_bylevel=1,colsample_bynode=1, 
                          colsample_bytree=0.8, gamma=0.4,max_delta_step=0,
                          learning_rate=0.1,min_child_weight=3, missing=None, 
                          n_estimators=800, n_jobs=1,random_state=0,reg_alpha=1e-05, 
                          reg_lambda=1, scale_pos_weight=1,subsample=0.8, verbosity=1)
  XGB.fit(X_train, y_train)
  score_train_acc = XGB.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = XGB.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_xgb = XGB.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_xgb))
  print("MSE value: ",mean_squared_error(y_val,y_pred_xgb))
  #print("Best Params", svm.best_params_)
  
  pkl_filename = "XGB_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(XGB, file)
  return score_train_acc, score_val_acc

def KNN_model(X_train, y_train,X_val,y_val):
  knn = KNeighborsRegressor(n_neighbors=5,weights='uniform',algorithm='auto')
  knn.fit(X_train, y_train)
  score_train_acc = knn.score(X_train, y_train)
  print("**** TRAINING INFORMATION ****")
  print("Training Accuracy: ", score_train_acc)
  score_val_acc = knn.score(X_val, y_val)
  print("Validation Accuracy: ", score_val_acc) 
  #y_train_check = LR.predict(X_train)
  print("------------------------------")
  print("------------------------------")
  print("**** Metrics and Scoring ****")
  y_pred_knn = knn.predict(X_val)
  print("R2 score: ",r2_score(y_val,y_pred_knn))
  print("MSE value: ",mean_squared_error(y_val,y_pred_knn))
  
  pkl_filename = "KNN_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(knn, file)
  return score_train_acc, score_val_acc

In [0]:
def train_val_model(models):
  print("The model user have provided for tarining:", models)
  learning_rate = 0.0001  # Used for Adam Optimizer
  iterr = 50 # Number of Epochs
  n_folds = 5 # Number of folds for Cross Validation
  bsize = 64  # Batch Size
  train_acc, val_acc,y_pred_values = [], [], [] # Lists for storing accuracies on each fold

  # Cross validation
  kfold = KFold(n_splits=n_folds, shuffle=True, random_state=7)
  hist = []
  val_hist=[]
  acc=[]
  val_acc=[]
  for train, val in kfold.split(X, y):
    X_train, X_val = X[train], X[val] # Grabbing X_train, X_val for Models
    y_train, y_val = y[train], y[val] # Grabbiing y_train, y_val for Models
    
    if models == "LR":
        lr_model = Linear_Regression_model(X_train, y_train,X_val,y_val)
        train_acc.append(lr_model[0])
        val_acc.append(lr_model[1])
        #print("TA",train_acc)
        print("Training Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)

    elif models == "LR_R":
        lr_model_r = Linear_Regression_Ridge_model(X_train, y_train,X_val,y_val)
        train_acc.append(lr_model_r[0])
        val_acc.append(lr_model_r[1])
        #print("TA",train_acc)
        print("**** VALIDATION INFORMATION ****")
        print("Training Accuracy During KFolds: ", sum(train_acc)/n_folds)
        print("Validation Accuracy During KFolds: ", sum(val_acc)/n_folds)
        
    elif models == "LR_L":
        lr_model_l = Linear_Regression_Lasso_model(X_train, y_train,X_val,y_val)
        train_acc.append(lr_model_l[0])
        val_acc.append(lr_model_l[1])
        #print("TA",train_acc)
        print("**** VALIDATION INFORMATION ****")
        print("Training Accuracy During KFolds: ", sum(train_acc)/n_folds)
        print("Validation Accuracy During KFolds: ", sum(val_acc)/n_folds)
        
    elif models == "LR_E":
        lr_model_e = Linear_Regression_ElasticNet_model(X_train, y_train,X_val,y_val)
        train_acc.append(lr_model_e[0])
        val_acc.append(lr_model_e[1])
        #print("TA",train_acc)
        print("**** VALIDATION INFORMATION ****")
        print("Training Accuracy During KFolds: ", sum(train_acc)/n_folds)
        print("Validation Accuracy During KFolds: ", sum(val_acc)/n_folds)

    elif models == "RF":
        rf_model = RF_model(X_train, y_train,X_val,y_val)
        train_acc.append(rf_model[0])
        val_acc.append(rf_model[1])
        y_pred_values.append(rf_model[2])
        #print("TA",train_acc)
        print("**** VALIDATION INFORMATION ****")
        print("Training Accuracy During KFolds: ", sum(train_acc)/n_folds)
        print("Validation Accuracy During KFolds: ", sum(val_acc)/n_folds)
        print("Y_pred Stats",y_pred_values)

    elif models == "GB":
        gb_model = GB_model(X_train, y_train,X_val,y_val)
        train_acc.append(gb_model[0])
        val_acc.append(gb_model[1])
        print("**** VALIDATION INFORMATION ****")
        print("Train Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)

    elif models == "XGB":
        xgb_model = XGBoost_model(X_train, y_train,X_val,y_val)
        train_acc.append(xgb_model[0])
        val_acc.append(xgb_model[1])
        print("**** VALIDATION INFORMATION ****")
        print("Train Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)

    elif models == "KNN":
        knn_model = KNN_model(X_train, y_train,X_val,y_val)
        train_acc.append(knn_model[0])
        val_acc.append(knn_model[1])
        print("**** VALIDATION INFORMATION ****")
        print("Train Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)
    

In [0]:
if __name__ == "__main__":
  model = "RF" # Depends on User, User can select models to experiment from the given list ["LR","LR_R","LR_L","LR_E","RF","GB","XGB","KNN"]
  _ = train_val_model(models=model)
  for el in _: print(el)

The model user have provided for tarining: RF


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


**** TRAINING INFORMATION ****
Training Accuracy:  0.9961272325805336
Validation Accuracy:  0.9778422228166128
------------------------------
------------------------------
**** Metrics and Scoring ****
Y_pred_RF values 0.4912652399505895
R2 score:  0.9778422228166128
MSE value:  0.004801914523504878
**** VALIDATION INFORMATION ****
Training Accuracy During KFolds:  0.1992254465161067
Validation Accuracy During KFolds:  0.19556844456332256
Y_pred Stats [array([0.04440625, 0.0578545 , 0.00609262, ..., 0.04673568, 0.02182668,
       0.00937902])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


**** TRAINING INFORMATION ****
Training Accuracy:  0.996077159730049
Validation Accuracy:  0.9741243100456439
------------------------------
------------------------------
**** Metrics and Scoring ****
Y_pred_RF values 0.5001350336696313
R2 score:  0.9741243100456439
MSE value:  0.00525515424534507
**** VALIDATION INFORMATION ****
Training Accuracy During KFolds:  0.3984408784621165
Validation Accuracy During KFolds:  0.3903933065724513
Y_pred Stats [array([0.04440625, 0.0578545 , 0.00609262, ..., 0.04673568, 0.02182668,
       0.00937902]), array([0.06790796, 0.01382512, 0.00222622, ..., 0.02230885, 0.00887408,
       0.00104389])]


...

**** TRAINING INFORMATION ****
Training Accuracy:  0.9961439735865503
Validation Accuracy:  0.9723820751515047
------------------------------
------------------------------
**** Metrics and Scoring ****
Y_pred_RF values 0.6793738392717603
R2 score:  0.9723820751515047
MSE value:  0.00546380942404356
**** VALIDATION INFORMATION ****
Training Accuracy During KFolds:  0.5976696731794265
Validation Accuracy During KFolds:  0.5848697216027523
Y_pred Stats [array([0.04440625, 0.0578545 , 0.00609262, ..., 0.04673568, 0.02182668,
       0.00937902]), array([0.06790796, 0.01382512, 0.00222622, ..., 0.02230885, 0.00887408,
       0.00104389]), array([0.01637075, 0.05354669, 0.00653148, ..., 0.00237118, 0.07320432,
       0.00518166])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


**** TRAINING INFORMATION ****
Training Accuracy:  0.9964397036325467
Validation Accuracy:  0.9691754146165132
------------------------------
------------------------------
**** Metrics and Scoring ****
Y_pred_RF values 0.5178071203720825
R2 score:  0.9691754146165132
MSE value:  0.005816152598520163
**** VALIDATION INFORMATION ****
Training Accuracy During KFolds:  0.796957613905936
Validation Accuracy During KFolds:  0.778704804526055
Y_pred Stats [array([0.04440625, 0.0578545 , 0.00609262, ..., 0.04673568, 0.02182668,
       0.00937902]), array([0.06790796, 0.01382512, 0.00222622, ..., 0.02230885, 0.00887408,
       0.00104389]), array([0.01637075, 0.05354669, 0.00653148, ..., 0.00237118, 0.07320432,
       0.00518166]), array([0.0374632 , 0.03192102, 0.0124722 , ..., 0.0186066 , 0.00308298,
       0.0740277 ])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


**** TRAINING INFORMATION ****
Training Accuracy:  0.9960825230632601
Validation Accuracy:  0.9757707169274458
------------------------------
------------------------------
**** Metrics and Scoring ****
Y_pred_RF values 0.6121282111701853
R2 score:  0.9757707169274458
MSE value:  0.005120573089659901
**** VALIDATION INFORMATION ****
Training Accuracy During KFolds:  0.996174118518588
Validation Accuracy During KFolds:  0.9738589479115441
Y_pred Stats [array([0.04440625, 0.0578545 , 0.00609262, ..., 0.04673568, 0.02182668,
       0.00937902]), array([0.06790796, 0.01382512, 0.00222622, ..., 0.02230885, 0.00887408,
       0.00104389]), array([0.01637075, 0.05354669, 0.00653148, ..., 0.00237118, 0.07320432,
       0.00518166]), array([0.0374632 , 0.03192102, 0.0124722 , ..., 0.0186066 , 0.00308298,
       0.0740277 ]), array([0.03659676, 0.05550533, 0.03028692, ..., 0.0147853 , 0.0011176 ,
       0.03482685])]


In [0]:
# Loading the trained pickled model
pkl_filename = "RF_pickle_model.pkl"
with open(pkl_filename, 'rb') as file:
  trained_model = pickle.load(file)

In [0]:
y_predictions = trained_model.predict(X_test)

In [0]:
y_predictions_train = trained_model.predict(X)

In [55]:
y_predictions_train

array([0.03659676, 0.07113874, 0.01887281, ..., 0.07815377, 0.00928875,
       0.00111356])

In [0]:
def calculate_wmae(y_predictions,y_truth,wmae_feature):
  wmae = np.zeros(y_predictions.shape)
  y_truth = y_truth.reshape(-1,1)
  #print(y_predictions.shape, y_train.shape)
  for i in range(len(y_predictions)):
    if wmae_feature[i] == 1:
      wmae[i] = 5*np.absolute(y[i]-y_predictions[i])
    else:
      wmae[i] = np.absolute(y[i]-y_predictions[i])
      
  wmae_feature = np.where(wmae_feature==1,5,wmae_feature)
  wmae_feature = np.where(wmae_feature==0,1,wmae_feature)
  return np.sum(wmae)/np.sum(wmae_feature)

In [57]:
print(calculate_wmae(y_predictions_train,y,wmae_feature))

0.0011948000090987031


In [0]:
y_actual_pred = min_max_y.inverse_transform(y_predictions.reshape(-1,1))

In [0]:
y_actual_pred.max()

508693.3886

In [0]:
X_test_for_submission = df_test_for_submission.drop("IsHoliday",axis=1).values

In [0]:
ret = np.column_stack((X_test_for_submission, y_actual_pred))

In [0]:
pd.DataFrame(ret).to_csv("Submission.csv", sep = ",", header = ["Store", "Department", "Datetriplet", "Weekley_Sales"], index = False)